# Aries Basic Controller Example
## Definitions Api

This can be used to write credential definitions to the ledger and search the definitions previously written by the agent.

A credential definition needs to be written to the ledger for every schema the agent wishes to issue. A credential definition acts as the public key in the [CL-signature scheme](https://wip-abramson.dev/cl-signatures) used in the sovrin ecosystem. 

For the interested the originally literature on this scheme can be found [here](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.186.5994&rep=rep1&type=pdf).

The key takeaway is that by using this type of signature scheme we are able to sign each attribute in the schema individual, making selective disclosure and efficient zero knowledge proof generation possible.

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)
    


## Write a Schema to the Ledger first

A credential definition is tied to a schema, each public DID may only write one credential definition per schema to the ledger. It is possible to write a credential definition for schemas that already exist on the ledger but because we are using a [fresh local network](http://localhost:9000) there aren't any so you must write one. See the schema_api notebook for more details.

In [3]:
# Define you schema name - must be unique on the ledger
schema_name = "my_new_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill"]

response = await agent_controller.schemas.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PQRXDxdGqQGSZ8z69p4xZP:2:my_new_schema:0.0.1


## Write a credential definition for this schema

#### Arguments
* schema_id - you need to pass in one of these and it must exist on the ledger
* tag - tag to identity the definition later, defaults to default
* support_revocation - Determines whether you will be able to later revoke the credential, defaults to False

**Note: You should be able to see the transactions on the local network [here](http://localhost:9000)**

In [4]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)


PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


## Get Created definition by id

As you can see it contains a bunch of BIG numbers all defined in the signature scheme. Notice that 'r' contains numbers for name, skill and master_secret, all attributes within the credential schema. (All credential schemas contain a master_secret to identify the holder)


In [5]:

definition = await agent_controller.definitions.get_by_id(cred_def_id)
print(definition)

{'credential_definition': {'ver': '1.0', 'id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schemaId': '10', 'type': 'CL', 'tag': 'default', 'value': {'primary': {'n': '117348249298555620816378097089215434370235308448936230955131133326654214868878217081815360006116132532953538129760584332613323908624443929256781050187007203845824068602545485327161105407309572114201236766832047249405780839144960309457027105204738426099677324035658127602685366142072443364571039078555128249206057310889689064821151714396593459219644050788423713316295200899057922618352787737845278926941967284999050097263244555827127035215351871104548689505480315236434790686926209663124464568110519549044459561305118753423638604455611699085584896532424877931651884708486784793523108647596658894160024453716368811205889', 's': '71561028554571956613466619459835273643120462810825345572292872886083817490237871032155864730769419512303268359310034954118885915991167092176072816557393326167513689021830229076519089775560675577410326866336

## Search all definitions the agent has created

#### Optional Arguments
* schema_id
* schema_issuer_did
* schema_version
* schema_name
* issuer_did
* cred_def_id

In [5]:
definitions = await agent_controller.definitions.search_created(cred_def_id=cred_def_id)
print(definitions)

{'credential_definition_ids': ['PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default']}
